In [53]:
import numpy as np
import time
import matplotlib.pyplot as plt
%matplotlib inline
import torch
import torch.optim as optim
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import open3d as o3d
import os
import torch
from torch.utils.data import Dataset, DataLoader
from dataclasses import dataclass
import sys
from pathlib import Path
import time
from pytorch3d.loss import chamfer_distance

import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import open3d as o3d
import os
import torch
from torch.utils.data import Dataset, DataLoader
from dataclasses import dataclass
import sys
from pathlib import Path
import time
from pytorch3d.loss import chamfer_distance

sys.path.append(str(Path.cwd().parent))

from Helpers.data import PointCloudDataset
import Helpers.PointCloudOpen3d as pc
from Helpers.data2 import PointCloudDataset2

if torch.cuda.is_available():
    device = "cuda"

elif hasattr(torch.backends, "mps") and torch.backends.mps.is_available():
    device = "mps"
else:
    device = "cpu"

print(f'Using: {device}')

Using: cuda


In [ ]:
point_size = 1024
# object_classes=['stool', 'chair', 'tv_stand', 'bench', 'desk', 'sofa', 'wardrobe', 'toilet', 'table', 'sink', 'night_stand', 'mantel','dresser', 'bookshelf','bed']
# object_classes=['stool', 'chair', 'bench', 'desk', 'table', 'night_stand', 'bookshelf']
# train_dataset = PointCloudDataset("../Data/ModelNet40", point_size, 'train', object_classes=['stool', 'chair', 'tv_stand', 'bench', 'desk'])
train_dataset = PointCloudDataset2("../Data/ModelNet40", point_size, 'train', object_classes=object_classes)
train_loader = DataLoader(train_dataset, batch_size = 64, shuffle = True)
print(len(train_dataset))


2316


In [84]:
test_dataset = PointCloudDataset2("../Data/ModelNet40", point_size, 'test', object_classes=object_classes)
test_loader = DataLoader(test_dataset, batch_size = 32, shuffle = True)
print(len(test_dataset))

426


In [178]:
import torch
from torch.utils.data import Dataset, DataLoader, random_split
import numpy as np

class ReadDataset(Dataset):
    def __init__(self,  source):
     
        self.data = torch.from_numpy(source).float()


    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        return self.data[index]

def RandomSplit(datasets, train_set_percentage):
    lengths = [int(len(datasets)*train_set_percentage), len(datasets)-int(len(datasets)*train_set_percentage)]
    return random_split(datasets, lengths)

def GetDataLoaders(npArray, batch_size, train_set_percentage = 0.9, shuffle=True, num_workers=0, pin_memory=True):
    
    
    pc = ReadDataset(npArray)

    train_set, test_set = RandomSplit(pc, train_set_percentage)

    train_loader = DataLoader(train_set, shuffle=shuffle, num_workers=num_workers, batch_size=batch_size, pin_memory=pin_memory)
    test_loader = DataLoader(test_set, shuffle=shuffle, num_workers=num_workers, batch_size=batch_size, pin_memory=pin_memory)
    
    return train_loader, test_loader

pc_array = np.load("../Data/chair_set.npy")

train_loader, test_loader = GetDataLoaders(npArray=pc_array, batch_size= 64)

In [55]:
x = next(iter(train_loader))[0]
cloud = pc.get_point_cloud(x)
pc.visualize_point_cloud(cloud)

KeyError: 0

In [125]:
feat = x.permute(0,2,1).clone()     # (3, 1024)
c1 = nn.Conv1d(3, 16,1) # (16, 1024) - each point has been blown up from 3 to 16 numbers  
c2 = nn.Conv1d(16, 32,1) # (32, 1024) - each point has been blown up from 16 to 32 numbers
c3 = nn.Conv1d(32, 32, 7, stride=1, padding=3) # (32, 1024) - each activation is now an amalgamation of 7 neighboring points 
c4 = nn.Conv1d(32, 32, 7, stride=2, padding=3) # (32, 512) - each activation is now an amalgamation of roughly 49 neighboring points (is this right? )
c5 = nn.Conv1d(32, 32, 7, stride=2, padding=3) # (32, 256)
c6 = nn.Conv1d(32, 32, 7, stride=2, padding=3) # (32, 128)

l1 = nn.Linear(4096, 2048)
l2 = nn.Linear(2048, 512)

o = c1(feat)
o = c2(o)
o = c3(o)
o = c4(o)
o = c5(o)
o = c6(o)
o = o.view(-1, 4096 )
o = l1(o)
lat = l2(o)

print(lat.shape)

torch.Size([64, 512])


In [99]:
dl1 = nn.Linear(512, 2048) # (2048)
dl2 = nn.Linear(2048, 4096) # (4096)
dc1 = nn.ConvTranspose1d(32, 32, 8, stride=2, padding = 3) # (32, 256)
dc2 = nn.ConvTranspose1d(32, 32, 8, stride=2, padding = 3) # (32, 512)
dc3 = nn.ConvTranspose1d(32, 32, 8, stride=2, padding= 3) # (32, 1024)
dc4 = nn.ConvTranspose1d(32, 32, 7, stride=1, padding= 3) # (32, 1024)
dc5 = nn.ConvTranspose1d(32, 16, 1) # (16, 1024)
dc6 = nn.ConvTranspose1d(16 ,3, 1) # (3, 1024)

o = dl1(lat)
o = dl2(o)
o = o.view(32, 128)
o = dc1(o)
o = dc2(o)
o = dc3(o)
o = dc4(o)
o = dc5(o)
o = dc6(o)

print(o.shape)

torch.Size([3, 1024])


In [4]:
class PointCloudAE(nn.Module):
    def __init__(self, point_size, latent_size):
        super(PointCloudAE, self).__init__()
        
        self.latent_size = latent_size
        self.point_size = point_size
        
        self.conv1 = torch.nn.Conv1d(3, 64, 1)
        self.conv2 = torch.nn.Conv1d(64, 128, 1)
        self.conv3 = torch.nn.Conv1d(128, self.latent_size, 1)
        self.bn1 = nn.BatchNorm1d(64)
        self.bn2 = nn.BatchNorm1d(128)
        self.bn3 = nn.BatchNorm1d(self.latent_size)
        
        self.dec1 = nn.Linear(self.latent_size,256)
        self.dec2 = nn.Linear(256,256)
        self.dec3 = nn.Linear(256,self.point_size*3)

    def encoder(self, x): 
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = self.bn3(self.conv3(x))
        x = torch.max(x, 2, keepdim=True)[0]
        x = x.view(-1, self.latent_size)
        return x
    
    def decoder(self, x):
        x = F.relu(self.dec1(x))
        x = F.relu(self.dec2(x))
        x = self.dec3(x)
        return x.view(-1, self.point_size, 3)
    
    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

In [ ]:
class ConvEncoder(nn.Module): 
    
    def __init__(self, latent_dim, point_size):
        super().__init__()


        self.c1 = nn.Conv1d(3, 64,1) # (16, point_size) - each point has been blown up from 3 to 16 numbers  
        self.c2 = nn.Conv1d(64, 128,1) # (32, point_size) - each point has been blown up from 16 to 32 numbers
        self.c3 = nn.Conv1d(128, 32, 7, stride=1, padding=3) # (32, point_size) - each activation is now an amalgamation of 7 neighboring points 
        self.c4 = nn.Conv1d(32, 32, 7, stride=2, padding=3) # (32, point_size / 2) - each activation is now an amalgamation of roughly 49 neighboring points (is this right? )
        self.c5 = nn.Conv1d(32, 32, 7, stride=2, padding=3) # (32, point_size / 4)
        self.c6 = nn.Conv1d(32, 32, 7, stride=2, padding=3) # (32, point_size / 8)

        self.lin7 = nn.Linear( int(point_size / 8) * 32, 2048)
        self.lin8 = nn.Linear(2048, latent_dim)

        self.bn1 = nn.BatchNorm1d(64)
        self.bn2 = nn.BatchNorm1d(128)
        self.bn3 = nn.BatchNorm1d(32)
        self.bn4 = nn.BatchNorm1d(32)
        self.bn5 = nn.BatchNorm1d(32)
        self.bn6 = nn.BatchNorm1d(32)
        self.bn7 = nn.BatchNorm1d(2048)

    def forward(self, x):
        x = F.gelu(self.bn1(self.c1(x)))
        x = F.gelu(self.bn2(self.c2(x)))
        x = F.gelu(self.bn3(self.c3(x)))
        x = F.gelu(self.bn4(self.c4(x)))
        x = F.gelu(self.bn5(self.c5(x)))
        x = F.gelu(self.bn6(self.c6(x)))

        x = x.view(-1, int(point_size / 8) * 32 )

        x = F.gelu(self.bn7(self.lin7(x)))
        x = self.lin8(x)

        return x

class ConvDecoder(nn.Module):
    
    def __init__(self, latent_dim, point_size):
        super().__init__()

        self.point_size = point_size

        self.dl1 = nn.Linear(latent_dim, 2048) # (2048)
        self.dl2 = nn.Linear(2048, int(point_size / 8) * 32) # (4096)

        self.dc3 = nn.ConvTranspose1d(32, 32, 8, stride=2, padding = 3) # (32, 256)
        self.dc4 = nn.ConvTranspose1d(32, 32, 8, stride=2, padding = 3) # (32, 512)
        self.dc5 = nn.ConvTranspose1d(32, 32, 8, stride=2, padding= 3) # (32, 1024)
        self.dc6 = nn.ConvTranspose1d(32, 128, 7, stride=1, padding= 3) # (32, 1024)
        self.dc7 = nn.ConvTranspose1d(128, 64, 1) # (16, 1024)
        self.dc8 = nn.ConvTranspose1d(64 ,3, 1) # (3, 1024)

        self.bn1 = nn.BatchNorm1d(2048)
        self.bn2 = nn.BatchNorm1d(4096)
        self.bn3 = nn.BatchNorm1d(32)
        self.bn4 = nn.BatchNorm1d(32)
        self.bn5 = nn.BatchNorm1d(32)
        self.bn6 = nn.BatchNorm1d(128)
        self.bn7 = nn.BatchNorm1d(64)

    def forward(self, x):
        
        x = F.gelu(self.bn1(self.dl1(x)))
        x = F.gelu(self.bn2(self.dl2(x)))

        x = x.view(-1, 32, int(self.point_size / 8))

        x = F.gelu(self.bn3(self.dc3(x)))
        x = F.gelu(self.bn4(self.dc4(x)))
        x = F.gelu(self.bn5(self.dc5(x)))
        x = F.gelu(self.bn6(self.dc6(x)))
        x = F.gelu(self.bn7(self.dc7(x)))
        x = F.gelu(self.dc8(x))

        return x

# class ConvDecoder(nn.Module):
    
#     def __init__(self, latent_dim, point_size):
#         super().__init__()
        
#         self.point_size = point_size

#         self.l1 = nn.Linear(latent_dim, 768)
#         self.l2 = nn.Linear(768 , 1024)
#         self.l3 = nn.Linear(1024, 2048)
#         self.l4 = nn.Linear(2048, 3072)
#         self.l5 = nn.Linear(3072, point_size * 3)

#     def forward(self, x):
#         x = F.gelu(self.l1(x))
#         x = F.gelu(self.l2(x))
#         x = F.gelu(self.l3(x))
#         x = F.gelu(self.l4(x))
#         x = self.l5(x)
#         x = x.view(-1, self.point_size, 3)
#         return x    
    
class ConvAutoEncoder(nn.Module):

    def __init__(self, latent_dim, point_size):
        super().__init__()

        self.encoder = ConvEncoder(latent_dim, point_size)
        self.decoder = ConvDecoder(latent_dim, point_size)

    def forward(self, x):
        latent_rep = self.encoder(x)
        recontructed_cloud = self.decoder(latent_rep)

        return recontructed_cloud


In [ ]:
# class PointCloudAE(nn.Module):
#     def __init__(self, point_size, latent_size):
#         super(PointCloudAE, self).__init__()
        
#         self.latent_size = latent_size
#         self.point_size = point_size
        
#         self.conv1 = torch.nn.Conv1d(3, 64, 1)
#         self.conv2 = torch.nn.Conv1d(64, 128, 1)
#         self.conv3 = torch.nn.Conv1d(128, 256, 1)
#         self.lin1 = nn.Linear(self.point_size, 512)
#         self.lin2 = nn.Linear(512, self.latent_size)
#         self.lin3 = nn.Linear(self.latent_size, 1)
#         self.bn1 = nn.BatchNorm1d(64)
#         self.bn2 = nn.BatchNorm1d(128)
#         self.bn3 = nn.BatchNorm1d(256)
#         # self.bn4 = nn.BatchNorm1d(self.latent_size)
        

#         # self.dec1 = nn.Linear(self.latent_size,self.point_size)
#         # self.dec2 = nn.Linear(self.point_size,2048)
#         # self.dec3 = nn.Linear(2048, 3072)
#         # self.dec4 = nn.Linear(3072,self.point_size*3)
#         self.l1 = nn.Linear(latent_size, 768)
#         self.l2 = nn.Linear(768 , 1024)
#         self.l3 = nn.Linear(1024, 2048)
#         self.l4 = nn.Linear(2048, 3072)
#         self.l5 = nn.Linear(3072, point_size * 3)

#     def encoder(self, x): 
#         x = F.relu(self.bn1(self.conv1(x)))
#         x = F.relu(self.bn2(self.conv2(x)))
#         x = F.relu(self.bn3(self.conv3(x)))
#         # x = F.relu(self.bn4(self.lin1(x)))
#         # x = F.relu(self.conv1(x))
#         # x = F.relu(self.conv2(x))
#         # x = F.relu(self.conv3(x))
#         x = F.relu(self.lin1(x))
#         x = F.relu(self.lin2(x))
#         x = self.lin3(x)
#         x = x.view(-1, self.latent_size)
#         return x
    
#     def decoder(self, x):
#         x = F.gelu(self.l1(x))
#         x = F.gelu(self.l2(x))
#         x = F.gelu(self.l3(x))
#         x = F.gelu(self.l4(x))
#         x = self.l5(x)
#         x = x.view(-1, self.point_size, 3)
#         return x    
    
#     def forward(self, x):
#         x = self.encoder(x)
#         x = self.decoder(x)
#         return x
    

In [70]:

# net = PointCloudAE2(point_size, 768).to(device)
# net = PointCloudAE(1024, 256).to(device)
# net = ConvAutoEncoder(512, 1024).to(device)
last_x = None

from pytorch3d.loss import chamfer_distance # chamfer distance for calculating point cloud distance

optimizer = optim.AdamW(net.parameters(), lr=0.0005)

def train_epoch():
    epoch_loss = 0
    for i, data in enumerate(train_loader):
        optimizer.zero_grad()
        
        x = data['points'].to(device)

        # x = data.to(device)
        output = net(x.permute(0,2,1)) # transpose data for NumberxChannelxSize format
        # print(x.shape, output.shape)
        loss, _ = chamfer_distance(x, output) 
        # loss = F.mse_loss(x, output)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        
    if i == 0:
        return epoch_loss / 1
    last_x = x.permute(0,2,1)
    return epoch_loss/i


train_loss_list = []  
test_loss_list = []  

for i in range(10000) :

    startTime = time.time()
    
    train_loss = train_epoch() #train one epoch, get the average loss
    train_loss_list.append(train_loss)

    
    epoch_time = time.time() - startTime
    
    if i % 100 == 0:
        print(f'Epoch {i} Train_loss: {train_loss}')

Epoch 0 Train_loss: 0.21064065614094338
Epoch 100 Train_loss: 0.004587396824111541
Epoch 200 Train_loss: 0.05521578429680732
Epoch 300 Train_loss: 0.04236642374760575
Epoch 400 Train_loss: 0.031202187968624964
Epoch 500 Train_loss: 0.019640171978001792
Epoch 600 Train_loss: 0.01035538508505043
Epoch 700 Train_loss: 0.0057365721376198865
Epoch 800 Train_loss: 0.0038459196221083403
Epoch 900 Train_loss: 0.003203008556738496
Epoch 1000 Train_loss: 0.0028051960398443043
Epoch 1100 Train_loss: 0.00195270146165664
Epoch 1200 Train_loss: 0.0014606259679163082
Epoch 1300 Train_loss: 0.001377678460105219
Epoch 1400 Train_loss: 0.001383946047604291
Epoch 1500 Train_loss: 0.0009551183983502495
Epoch 1600 Train_loss: 0.0029989111482993597
Epoch 1700 Train_loss: 0.0021191561213021893
Epoch 1800 Train_loss: 0.0014566939369413173
Epoch 1900 Train_loss: 0.001128835551854637
Epoch 2000 Train_loss: 0.0010564515186059806
Epoch 2100 Train_loss: 0.000891223844114898
Epoch 2200 Train_loss: 0.000930713649722

KeyboardInterrupt: 

In [27]:
torch.save(net.state_dict(), 'trained_models/furniture_10k')

In [49]:
torch.save(net, 'trained_models/furniture_model_weights_50k')

In [50]:
model = torch.load('trained_models/furniture_model_weights_50k', weights_only = False)

None


In [98]:
# x = next(iter(train_loader))['points']

# cloud = pc.get_point_cloud(x[0])
# pc.visualize_point_cloud(cloud)

# with torch.no_grad():
#     data = x.permute(0,2,1).to(device)
#     rec_x = np.array(net(data)[0].to('cpu'))
#     cloud = pc.get_point_cloud(rec_x)
#     pc.visualize_point_cloud(cloud)

x = next(iter(test_loader))['points']

cloud = pc.get_point_cloud(x[0])
pc.visualize_point_cloud(cloud)

with torch.no_grad():
    data = x.permute(0,2,1).to(device)
    rec_x = np.array(net(data)[0].to('cpu'))
    cloud = pc.get_point_cloud(rec_x)
    pc.visualize_point_cloud(cloud)

In [62]:
model = net
x = next(iter(test_loader))['points'][0]

cloud = pc.get_point_cloud(x)
pc.visualize_point_cloud(cloud)

with torch.no_grad():
    data = x.unsqueeze(0).permute(0,2,1).to(device)

    rec_x = np.array(model(data)[0].to('cpu'))
    cloud = pc.get_point_cloud(rec_x)
    pc.visualize_point_cloud(cloud)

ValueError: Expected more than 1 value per channel when training, got input size torch.Size([1, 2048])

In [41]:

x = next(iter(train_loader))['points'][0][0:50]
cloud = pc.get_point_cloud(x)
pc.visualize_point_cloud(cloud)


In [100]:
x_norm


tensor([[0.5825, 0.5014, 0.1379],
        [0.5915, 0.4976, 0.1370],
        [0.8215, 0.3296, 0.1172],
        ...,
        [0.6037, 0.3774, 0.0789],
        [0.6028, 0.3761, 0.0731],
        [0.6030, 0.3761, 0.0733]])

In [ ]:
point_size = 768
train_dataset = PointCloudDataset("../Data/ModelNet40", point_size, 'train', object_classes=['stool', 'chair', 'tv_stand', 'bench', 'desk'])
train_loader = DataLoader(train_dataset, batch_size = 64, shuffle = True)

class PointCloudAE(nn.Module):
    def __init__(self, point_size, latent_size):
        super(PointCloudAE, self).__init__()
        
        self.latent_size = latent_size
        self.point_size = point_size
        
        self.conv1 = torch.nn.Conv1d(3, 64, 1)
        self.conv2 = torch.nn.Conv1d(64, 128, 1)
        self.conv3 = torch.nn.Conv1d(128, 256, 1)
        self.lin1 = nn.Linear(self.point_size, self.latent_size)
        self.lin2 = nn.Linear(self.latent_size, 1)
        # self.bn1 = nn.BatchNorm1d(128)
        # self.bn2 = nn.BatchNorm1d(256)
        # self.bn3 = nn.BatchNorm1d(256)
        # self.bn4 = nn.BatchNorm1d(self.latent_size)
        

        self.dec1 = nn.Linear(self.latent_size,self.point_size)
        self.dec2 = nn.Linear(self.point_size,2048)
        self.dec3 = nn.Linear(2048, 3072)
        self.dec4 = nn.Linear(3072,self.point_size*3)

    def encoder(self, x): 
        # x = F.relu(self.bn1(self.conv1(x)))
        # x = F.relu(self.bn2(self.conv2(x)))
        # x = F.relu(self.bn3(self.conv3(x)))
        # x = F.relu(self.bn4(self.lin1(x)))
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = F.relu(self.lin1(x))
        x = self.lin2(x)
        x = x.view(-1, self.latent_size)
        return x
    
    def decoder(self, x):
        x = F.relu(self.dec1(x))
        x = F.relu(self.dec2(x))
        x = F.relu(self.dec3(x))
        x = self.dec4(x)
        x = x.view(-1, self.point_size, 3)
        return x
    
    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x
    